## Determine which professionals Vanderbilt Hospital should reach out to in the Nashville area to expand their own patient volume.
## First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).
## Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?

In [2]:
import sqlite3
import pandas as pd
#pd.set_option('display.max_rows', None)

### Produce a list of Hospitals with 'Vanderbilt' in the organization name in Nashville TN, not every organization from this list is in Hop table

In [49]:
query = '''
SELECT npi, `provider_organization_name_(legal_business_name)`, 
`provider_business_practice_location_address_city_name`, COUNT(*)
FROM nppes
WHERE `provider_organization_name_(legal_business_name)` LIKE '%Vanderbilt%' 
AND `provider_business_practice_location_address_state_name` = 'TN'
GROUP BY `provider_organization_name_(legal_business_name)`;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    test_sqlite = pd.read_sql(query, db)
    
test_sqlite

# Not all organizations from this list appear in HOP dataset

,npi,provider_organization_name_(legal_business_name),provider_business_practice_location_address_city_name,COUNT(*)
0,1891951513,CONCENTRA VANDERBILT LLC,MURFREESBORO,5
1,1881885960,"DOUGLAS L. VANDERBILT, M.D. ,P.C.",CHATTANOOGA,1
2,1255577466,MONROE CARELL JR VANDERBILT CHILDREN'S HOSPITAL,NASHVILLE,1
3,1003211145,MONROE CARELL JR. CHILDREN'S HOSPITAL AT VANDE...,NASHVILLE,1
4,1578646006,VANDERBILT ASTHMA SINUS ALLERGY PROGRAM,NASHVILLE,1
5,1134306244,"VANDERBILT ASTHMA SINUS AND ALLERGY PROGRAM, LLC",NASHVILLE,1
6,1649860305,"VANDERBILT BEDFORD HOSPITAL, LLC",SHELBYVILLE,9
7,1609056399,VANDERBILT CHILDREN'S,NASHVILLE,1
8,1942757307,VANDERBILT CHILDREN'S HOSPITAL PHARMACY,NASHVILLE,1
9,1326224015,VANDERBILT CHILDRENS HOSPITAL,NASHVILLE,1


### We learned that 'VANDERBILT UNIVERSITY MEDICAL CENTER' has the most patient referrals from previous tasks. 
### Moreover, 'VANDERBILT UNIVERSITY MEDICAL CENTER' has MANY distinct NPI #. So the next step is to store all npi with organization name 'VANDERBILT UNIVERSITY MEDICAL CENTER' in a list

In [3]:
# Add 'OR `provider_organization_name_(legal_business_name)` = 'SHELBYVILLE HOSPITAL COMPANY LLC' in the WHERE clause if true
query = '''
SELECT GROUP_CONCAT(DISTINCT npi) AS Vanderbilt_npi
FROM nppes
WHERE `provider_organization_name_(legal_business_name)` = 'VANDERBILT UNIVERSITY MEDICAL CENTER'
AND `provider_business_practice_location_address_state_name` = 'TN'
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    npi_concat_sqlite = pd.read_sql(query, db)
    
Vanderbilt_npi = npi_concat_sqlite.iloc[0][0]
Vanderbilt_npi

'1477709053,1245485200,1366679276,1073833778,1528388196,1801161708,1508132242,1427394634,1336560598,1265833487,1487029997,1902261860,1659738383,1932352283,1912443326,1063958270,1700800877,1538670179,1679971485,1649637794,1013553155,1700950284,1972711828,1285091330,1548920465,1053449660,1144857160,1194186700,1215078027,1215979190,1306889597,1396882205,1558408633,1821030842,1104202761,1770944464,1972027159,1740319847'

## First, research which professionals are sending significant numbers of patients only to competitor hospitals (such as TriStar Centennial Medical Center).

In [4]:
# Get NONE Vandy providers INNER JOIN CCN #
query = f'''
WITH Vandy_providers AS (
    SELECT DISTINCT hop.from_npi
    FROM hop_major_TN_hospitals AS hop
    WHERE hop.to_npi IN ({Vanderbilt_npi}))
SELECT hop.from_npi, hos1.facility_name AS from_facility_affiliation, SUM(patient_count) AS total_patients
FROM hop_major_TN_hospitals AS hop
JOIN nppes AS n
ON n.npi = hop.from_npi
LEFT JOIN facility_affiliations AS fa1
ON hop.from_npi = fa1.npi
LEFT JOIN TN_Hospital_Info AS hos1
ON fa1.facility_afl_ccn = hos1.facility_id
WHERE hop.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY hop.from_npi
ORDER BY total_patients DESC;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_sqlite_test = pd.read_sql(query, db)
None_Vandy_sqlite_test.head()


,from_npi,from_facility_affiliation,total_patients
0,1417131715,VANDERBILT UNIVERSITY MEDICAL CENTER,31800
1,1912984758,None,23500
2,1871548818,TRISTAR SKYLINE MEDICAL CENTER,23045
3,1306993282,None,21670
4,1376756742,JACKSON-MADISON COUNTY GENERAL HOSPITAL,20410


In [50]:
# Get NONE Vandy providers
query = f'''
WITH Vandy_providers AS (
    SELECT DISTINCT hop.from_npi
    FROM hop_major_TN_hospitals AS hop
    WHERE hop.to_npi IN ({Vanderbilt_npi}))
SELECT hop.from_npi, SUM(patient_count) AS total_patients, n.*
FROM hop_major_TN_hospitals AS hop
JOIN nppes AS n
ON n.npi = hop.from_npi
WHERE hop.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY hop.from_npi
ORDER BY total_patients DESC
LIMIT 100;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_sqlite = pd.read_sql(query, db)
None_Vandy_sqlite


,from_npi,total_patients,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,1417131715,6360,1417131715,1.0,None,RIDDICK,JOHN,ALSTON,DR.,None,M.D.,2400 PATTERSON ST,SUITE 502,NASHVILLE,TN,37203,207RI0011X
1,1912984758,4700,1912984758,1.0,None,LASSITER,GREGORY,L,DR.,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
2,1871548818,4609,1871548818,1.0,None,KLEIN,WILLIAM,J,DR.,None,MD,7945 WOLF RIVER BLVD,None,GERMANTOWN,TN,38138,2085R0202X
3,1306993282,4334,1306993282,1.0,None,SHIPMAN,JASON,None,None,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
4,1376756742,4082,1376756742,1.0,None,GRIFFIN,BENJAMIN,DAVID,None,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
5,1447234141,3819,1447234141,1.0,None,CAPLAN,STUART,H,DR.,None,MD,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
6,1821060526,3797,1821060526,1.0,None,WATERS,RONALD,DOUGLAS,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
7,1073531042,3768,1073531042,1.0,None,GRIPPO,JAMES,DAVID,DR.,None,M.D.,1224 TROTWOOD AVE,None,COLUMBIA,TN,38401,2085R0202X
8,1487816948,3752,1487816948,1.0,None,PATTERSON,GARY,TRAVIS,DR.,None,M.D.,210 25TH AVE N STE 1204,None,NASHVILLE,TN,37203,2085R0202X
9,1710932017,3735,1710932017,1.0,None,HIMMELFARB,ELLIOT,H,MR.,None,M.D.,3024 BUSINESS PARK CIR,None,GOODLETTSVILLE,TN,37072,2085R0202X


## Next, consider the specialty of the provider. If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors who should they reach out to in those areas?

In [70]:
# Group NONE Vandy providers by specialization
query = f'''
SELECT specialization, total_patients, ROUND(specialization_referral_ratio*100,2) AS referral_pct_total, 
    hos1.facility_name AS from_facility_affiliation, from_df.npi, 
    from_df.`provider_last_name_(legal_name)`, from_df.provider_middle_name, from_df.provider_first_name
FROM (
WITH specialization AS (
    SELECT t.specialization, SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
    FROM hop_major_TN_hospitals AS h
    INNER JOIN nppes n1
    ON h.from_npi = n1.npi
    INNER JOIN taxonomy as t
    ON n1.primary_taxonomy = t.code
    WHERE h.from_npi NOT IN (SELECT * FROM Vandy_providers)
    GROUP BY t.specialization
),

Vandy_providers AS (
    SELECT DISTINCT h.from_npi
    FROM hop_major_TN_hospitals AS h
    WHERE h.to_npi IN ({Vanderbilt_npi}))
    
SELECT t. specialization, SUM(patient_count) AS total_patients, 
       s.total_patients AS specialization_total_patients_ref,
       SUM(h.patient_count)*1.0/s.total_patients AS specialization_referral_ratio,
       n.*
FROM hop_major_TN_hospitals AS h
JOIN nppes AS n
ON n.npi = h.from_npi
JOIN taxonomy AS t
ON t.code = n.primary_taxonomy
INNER JOIN specialization AS s
ON s.specialization = t.specialization 
WHERE h.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY h.from_npi
ORDER BY t.specialization, total_patients DESC) AS from_df
LEFT JOIN facility_affiliations AS fa1
ON from_df.npi = fa1.npi
LEFT JOIN TN_Hospital_Info AS hos1
ON fa1.facility_afl_ccn = hos1.facility_id;
'''




with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_byspecialization_affiliation = pd.read_sql(query, db)
None_Vandy_byspecialization_affiliation.head(50)

,specialization,total_patients,referral_pct_total,from_facility_affiliation,npi,provider_last_name_(legal_name),provider_middle_name,provider_first_name
0,Acute Care,555,5.69,SAINT THOMAS RUTHERFORD HOSPITAL,1649718230,GRABOVICH,None,DANIELLE
1,Acute Care,526,5.40,TRISTAR CENTENNIAL MEDICAL CENTER,1487041513,NEUFELD,DANIEL,LUCAS
2,Acute Care,388,3.98,SAINT THOMAS WEST HOSPITAL,1427334176,GRAUZER,WILLIAM,FRANKLIN
3,Acute Care,347,3.56,SAINT THOMAS RUTHERFORD HOSPITAL,1508892688,PEMBRIDGE,L,RICHARD
4,Acute Care,347,3.56,SAINT THOMAS WEST HOSPITAL,1508892688,PEMBRIDGE,L,RICHARD
5,Acute Care,331,3.40,SAINT THOMAS WEST HOSPITAL,1811258213,HILL,WALDROP,KIMBERLY
6,Acute Care,328,3.36,None,1750747309,PINKSTON,None,JOSHUA
7,Acute Care,323,3.31,SAINT THOMAS WEST HOSPITAL,1568784205,MILAM,J,DANIEL
8,Acute Care,295,3.03,None,1487788188,ROACH,NICHOLAS,ANDREW
9,Acute Care,279,2.86,None,1457628265,SCHALLENBERGER,None,KAITLYN


## Solution

If Vanderbilt wants to increase volume from Orthopedic Surgeons or from Family Medicine doctors, they may consider reaching out to family medicine providers in top 5 affiliations (see result below) and Orthopedic Surgeons from Saint Thomas and Tristar networks. 

In [52]:
# Group NONE Vandy providers by specialization
query = f'''
SELECT specialization, from_facility_affiliation, COUNT(npi) AS providers_count
FROM (
SELECT specialization, total_patients, ROUND(specialization_referral_ratio*100,2) AS referral_pct_total, 
    hos1.facility_name AS from_facility_affiliation, from_df.npi, 
    from_df.`provider_last_name_(legal_name)`, from_df.provider_middle_name, from_df.provider_first_name
FROM (
WITH specialization AS (
    SELECT t.specialization, SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
    FROM hop_major_TN_hospitals AS h
    INNER JOIN nppes n1
    ON h.from_npi = n1.npi
    INNER JOIN taxonomy as t
    ON n1.primary_taxonomy = t.code
    GROUP BY t.specialization
),

Vandy_providers AS (
    SELECT DISTINCT h.from_npi
    FROM hop_major_TN_hospitals AS h
    WHERE h.to_npi IN ({Vanderbilt_npi}))
    
SELECT t. specialization, SUM(patient_count) AS total_patients, 
       s.total_patients AS specialization_total_patients_ref,
       SUM(h.patient_count)*1.0/s.total_patients AS specialization_referral_ratio,
       n.*
FROM hop_major_TN_hospitals AS h
JOIN nppes AS n
ON n.npi = h.from_npi
JOIN taxonomy AS t
ON t.code = n.primary_taxonomy
INNER JOIN specialization AS s
ON s.specialization = t.specialization 
WHERE h.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY h.from_npi
ORDER BY t.specialization, total_patients DESC) AS from_df
LEFT JOIN facility_affiliations AS fa1
ON from_df.npi = fa1.npi
LEFT JOIN TN_Hospital_Info AS hos1
ON fa1.facility_afl_ccn = hos1.facility_id)
GROUP BY specialization, from_facility_affiliation;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_byspecialization_affiliation_count = pd.read_sql(query, db)
None_Vandy_byspecialization_affiliation_count

,specialization,from_facility_affiliation,providers_count
0,Acute Care,None,26
1,Acute Care,BLOUNT MEMORIAL HOSPITAL,1
2,Acute Care,MAURY REGIONAL HOSPITAL,1
3,Acute Care,METRO NASHVILLE GENERAL HOSPITAL,1
4,Acute Care,RIVERVIEW REGIONAL MEDICAL CENTER,1
...,...,...,...
614,Women's Health,SAINT THOMAS WEST HOSPITAL,3
615,Women's Health,SOUTHERN TENNESSEE REGIONAL HEALTH SYSTEM LAWR...,1
616,Women's Health,TRISTAR CENTENNIAL MEDICAL CENTER,1
617,Women's Health,VANDERBILT UNIVERSITY MEDICAL CENTER,1


In [53]:
None_Vandy_byspecialization_affiliation_count.to_csv('None_Vandy_byspecialization_affiliation_count.csv')

In [28]:
# Group NONE Vandy providers by specialization
query = f'''
WITH specialization AS (
    SELECT t.specialization, SUM(h.transaction_count) AS total_transactions, SUM(h.patient_count) AS total_patients
    FROM hop_major_TN_hospitals AS h
    INNER JOIN nppes n1
    ON h.from_npi = n1.npi
    INNER JOIN taxonomy as t
    ON n1.primary_taxonomy = t.code
    WHERE h.from_npi NOT IN (SELECT * FROM Vandy_providers)
    GROUP BY t.specialization
),

Vandy_providers AS (
    SELECT DISTINCT h.from_npi
    FROM hop_major_TN_hospitals AS h
    WHERE h.to_npi IN ({Vanderbilt_npi}))
    
SELECT t. specialization, SUM(patient_count) AS total_patients, 
       s.total_patients AS specialization_total_patients_ref,
       SUM(h.patient_count)*1.0/s.total_patients AS specialization_referral_ratio,
       n.*
FROM hop_major_TN_hospitals AS h
JOIN nppes AS n
ON n.npi = h.from_npi
JOIN taxonomy AS t
ON t.code = n.primary_taxonomy
INNER JOIN specialization AS s
ON s.specialization = t.specialization 
WHERE h.from_npi NOT IN (SELECT * FROM Vandy_providers)
GROUP BY h.from_npi
ORDER BY t.specialization, total_patients DESC ;
'''


with sqlite3.connect('Hop_Teaming_2018.sqlite') as db: 
    None_Vandy_byspecialization_sqlite = pd.read_sql(query, db)
None_Vandy_byspecialization_sqlite

,specialization,total_patients,specialization_total_patients_ref,specialization_referral_ratio,npi,entity_type_code,provider_organization_name_(legal_business_name),provider_last_name_(legal_name),provider_first_name,provider_middle_name,provider_name_prefix_text,provider_name_suffix_text,provider_credential_text,provider_first_line_business_practice_location_address,provider_second_line_business_practice_location_address,provider_business_practice_location_address_city_name,provider_business_practice_location_address_state_name,provider_business_practice_location_address_postal_code,primary_taxonomy
0,Acute Care,555,9749,0.056929,1649718230,1.0,None,GRABOVICH,DANIELLE,None,None,None,APN,1840 MEDICAL CENTER PKWY STE 201,None,MURFREESBORO,TN,37129,363LA2100X
1,Acute Care,526,9749,0.053954,1487041513,1.0,None,NEUFELD,LUCAS,DANIEL,None,None,AGACNP-BC,2300 PATTERSON ST,None,NASHVILLE,TN,37203,363LA2100X
2,Acute Care,388,9749,0.039799,1427334176,1.0,None,GRAUZER,FRANKLIN,WILLIAM,None,None,APN,4230 HARDING RD,SUITE 435,NASHVILLE,TN,37205,363LA2100X
3,Acute Care,347,9749,0.035593,1508892688,1.0,None,PEMBRIDGE,RICHARD,L,None,JR.,NP,2010 CHURCH ST,SUITE 201,NASHVILLE,TN,37203,363LA2100X
4,Acute Care,331,9749,0.033952,1811258213,1.0,None,HILL,KIMBERLY,WALDROP,MRS.,None,NP,4230 HARDING PIKE,SUITE 503,NASHVILLE,TN,37205,363LA2100X
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1231,Women's Health,135,1551,0.087041,1487780268,1.0,None,KRANTZ,NORMA,J,None,None,N.P.,300 20TH AVE N STE 401,None,NASHVILLE,TN,37203,364SW0102X
1232,Women's Health,106,1551,0.068343,1780848010,1.0,None,NEWMAN,ELLEN,B,None,None,RN MSN WHNP,2011 MURPHY AVE STE 601,None,NASHVILLE,TN,37203,363LW0102X
1233,Women's Health,80,1551,0.051580,1336416189,1.0,None,STALEY,JULIA,CHRISTINE,None,None,"MSN, WHNP-BC",820 HATCHER LN,None,COLUMBIA,TN,38401,363LW0102X
1234,Women's Health,71,1551,0.045777,1801068010,1.0,None,GORDON,LEANN,D,MS.,None,NP,808 JENLAND DR,None,COLUMBIA,TN,38401,364SW0102X


In [51]:
competitors_npi_diag_radio = None_Vandy_byspecialization_sqlite[None_Vandy_byspecialization_sqlite['specialization'] == 'Diagnostic Radiology'][['specialization','total_patients','specialization_referral_ratio','npi', 'provider_last_name_(legal_name)', 'provider_first_name', 'provider_middle_name']]
competitors_npi_diag_radio.head()

,specialization,total_patients,specialization_referral_ratio,npi,provider_last_name_(legal_name),provider_first_name,provider_middle_name
254,Diagnostic Radiology,4700,0.018057,1912984758,LASSITER,GREGORY,L
255,Diagnostic Radiology,4609,0.017708,1871548818,KLEIN,WILLIAM,J
256,Diagnostic Radiology,4334,0.016651,1306993282,SHIPMAN,JASON,None
257,Diagnostic Radiology,4082,0.015683,1376756742,GRIFFIN,BENJAMIN,DAVID
258,Diagnostic Radiology,3819,0.014673,1447234141,CAPLAN,STUART,H


In [48]:
competitors_npi_diag_radio.to_csv('competitors_npi_diag_radio.csv')

In [39]:
None_Vandy_byspecialization_sqlite.to_csv('competitors_specialization.csv')

In [29]:
None_Vandy_byspecialization_sqlite[None_Vandy_byspecialization_sqlite['specialization'] == 'Acute Care']['total_patients'].sum()

9749

In [17]:
None_Vandy_byspecialization_sqlite['npi'].nunique()

1236

In [44]:
None_Vandy_byspecialization_sqlite.groupby('specialization')['total_patients'].sum().head(20)

specialization
Acute Care                                            9749
Addiction Medicine                                     255
Adolescent Medicine                                     30
Adult Health                                          3505
Adult Medicine                                         971
Adult Reconstructive Orthopaedic Surgery               916
Advanced Heart Failure and Transplant Cardiology      4814
Allergy                                                226
Anatomic Pathology & Clinical Pathology              23736
Brain Injury Medicine                                   84
Cardiovascular Disease                               63512
Clinical                                               317
Clinical Cardiac Electrophysiology                    7227
Clinical Child & Adolescent                             36
Clinical Pathology/Laboratory Medicine                 626
Counseling                                              55
Critical Care Medicine                   

In [34]:
None_Vandy_byspecialization_sqlite['total_patients'].sum()

572555

In [35]:
None_Vandy_byspecialization_sqlite[None_Vandy_byspecialization_sqlite['specialization'] == 'Family'][['specialization_referral_ratio','provider_first_name','provider_last_name_(legal_name)']].head()


,specialization_referral_ratio,provider_first_name,provider_last_name_(legal_name)
422,0.022483,TIMOTHY,DICKEY
423,0.015743,JILLIAN,DIMOND
424,0.015589,KERI,GALBREATH
425,0.015332,TINA,DIEROLF
426,0.015023,LINDSEY,CHAPMAN


In [36]:
None_Vandy_byspecialization_sqlite[None_Vandy_byspecialization_sqlite['specialization'] == 'Orthopaedic Surgery of the Spine'][['specialization_referral_ratio','provider_first_name','provider_last_name_(legal_name)']].head()

,specialization_referral_ratio,provider_first_name,provider_last_name_(legal_name)
967,0.183173,JOHN,KLEKAMP
968,0.155127,MICHAEL,MCNAMARA
969,0.140520,ERION,QAMIRANI
970,0.103126,LAWRENCE,BABAT
971,0.075372,JAMES,FISH


In [37]:
d = None_Vandy_byspecialization_sqlite.groupby('specialization')['npi'].count().sort_values(ascending=False)
df = pd.DataFrame(data=d) 
df

,npi
specialization,
Family,286
Diagnostic Radiology,143
Cardiovascular Disease,65
Acute Care,60
Pulmonary Disease,55
...,...
Orthopedic,1
Otology & Neurotology,1
Pediatric Cardiology,1


In [72]:
df.to_csv('competitors_specialization.csv')

In [38]:
len(None_Vandy_byspecialization_sqlite.groupby('specialization')['npi'].count().sort_values(ascending=False))

81